In [1]:
import pandas as pd
import numpy as np
import torch
import networkx as nx
import pickle

from biopandas.pdb import PandasPdb
from torch_geometric.data import Data
from torch_geometric.utils import convert
from rdkit import Chem
from rdkit.Chem import AllChem
from torchdrug import data, utils
from graphein.protein.graphs import construct_graph
from tqdm import tqdm

/Users/yoshitakainoue/.pyenv/versions/miniforge3-4.10.1-5/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[11/23/23 14:10:55] WARNING  To use the Graphein submodule                                         ]8;id=188305;file:///Users/yoshitakainoue/.pyenv/versions/miniforge3-4.10.1-5/envs/py310/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=523390;file:///Users/yoshitakainoue/.pyenv/versions/miniforge3-4.10.1-5/envs/py310/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py#44\44]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             biovec cannot be installed via conda                                                  
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

                    WARNING  To use the Graphein submodule graphein.protein.visualisation, you  ]8;id=641306;file:///Users/yoshitakainoue/.pyenv/versions/miniforge3-4.10.1-5/envs/py310/lib/python3.10/site-packages/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=10811;file:///Users/yoshitakainoue/.pyenv/versions/miniforge3-4.10.1-5/envs/py310/lib/python3.10/site-packages/graphein/protein/visualisation.py#35\35]8;;\
                             need to install: pytorch3d                                                            
                             To do so, use the following command: conda install -c pytorch3d                       
                             pytorch3d                                                                             

                    WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=853850;file:///Users/yoshitakainoue/.pyenv/versions/miniforge3-4.10.1-5/envs/py310/lib/python3.10/site-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=617489;file:///Users/yoshitakainoue/.pyenv/versions/miniforge3-4.10.1-5/envs/py310/lib/python3.10/site-packages/graphein/protein/meshes.py#29\29]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: conda install -c pytorch3d pytorch3d             

In [2]:
def get_protein_graph(i):
    atom_df = PandasPdb().read_pdb(i).df['ATOM']
    node_feature = atom_df[['residue_number', 'occupancy', 'b_factor',]]
    position = atom_df[['x_coord', 'y_coord', 'z_coord']]

    mol = Chem.MolFromPDBFile(i)
    atoms = [(atom.GetFormalCharge(), atom.GetHybridization()) for atom in mol.GetAtoms()]
    nodes = pd.concat([pd.DataFrame(atoms), node_feature], axis=1)

    atom_type = np.array([a.GetAtomicNum() for a in mol.GetAtoms()])

    bonds = [(
        bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond.GetBondType(), bond.GetStereo(), 
    ) for bond in mol.GetBonds()]
    bonds = pd.DataFrame(bonds).values

    edges = bonds[:, :2]
    edges_attr = bonds[:, 2:]

    node_features = torch.Tensor(nodes.values.astype(int))
    atom_type = torch.Tensor(atom_type.astype(int))
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edges_attr = torch.tensor(edges_attr, dtype=torch.long)
    position = torch.tensor(position.values, dtype=torch.long)
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edges_attr, pos=position, atom_type=atom_type)

    return data

In [3]:
proteins = !ls pdb/

In [4]:
len(proteins)

9624

In [5]:
protein_graphs = {}
errors = []
for i in proteins:
    try:
        protein_graphs[i.split('.')[0]] = get_protein_graph('pdb/'+i)
    except:
        print('###')
        errors.append(i)

[14:11:05] Explicit valence for atom # 17910 O, 3, is greater than permitted


###


[14:11:24] Explicit valence for atom # 1081 O, 3, is greater than permitted


###


[14:11:25] Explicit valence for atom # 7102 O, 3, is greater than permitted


...

###


[14:25:55] Explicit valence for atom # 534 O, 3, is greater than permitted


###


[14:26:44] Explicit valence for atom # 8737 O, 3, is greater than permitted


###


In [6]:
len(errors)

43

In [10]:
import numpy as np
# !mkdir protein_graphs

In [11]:
for i in protein_graphs.keys():
    torch.save(protein_graphs[i], 'protein_graphs/{}.pt'.format(i))

In [15]:
protein_graphs['A0A024RBG1']

Data(x=[1437, 5], edge_index=[2, 1467], edge_attr=[1467, 2], pos=[1437, 3], atom_type=[1437])